# Wrangling OSM Data.
---

## Map Area 
Cincinnati, OH, USA
* [Map zen](https://mapzen.com/data/metro-extracts/your-extracts/91001a3b94af)
* [Open Street Map](https://www.openstreetmap.org/relation/183453#map=12/39.1369/-84.5405)

I'm currently an international student at University of Cincinnati and thus I chose this city to see what I get from it.

---

# Problems
* **There was no problematic Character **:

  ```python
  {'problemchars': 0, 'lower': 124871, 'other': 1866, 'lower_colon': 45631} ```
  
* **The over abbreviated / lower case street names were:**
     ```python
     {'st', 'Rd', 'Ave', 'avenue'} ```
* **Not expected street types :**
     ```python
     {'Way', 'Plaza', 'Circle', 'Hill','Warner'}```
* **There were some zip codes from outside the city; Cincinnati start with 452 :**
     ```python 
     ['41073', '41071', '41075', '41074', '41011','45150']```
* **Some zip codes were written in a wrong way :**
   ```python
   ['45220-1405', '45223-1806','45208-2101','45220-1129', '45208-2017'] ```





# Fixing the Problem 

1. ** I fixed the shortcut by updating the mapping dictionary**
``` python
# Fixing shortcuts/ lower cases
mapping = { "St": "Street",
            'Ave': "Avenue",
            "avenue":"Avenue",
            'Rd.':'Road'
            }
def update_name(name, mapping):
    for k, v in mapping.iteritems():
        for word in name.split():
            if word == k:
                name = name.replace(k,v,1)
                break
        return name 
      ```
2. **I added the unexpected types to my checking list**
 ``` python       
# 2. Expected list updated 
Expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Plaza","Hill",'Ludlow','Way','Circle','Warner']        
```
3. **I Fixed the zip codes by taking only the first five characters **
``` python
# 3.  Fixing the zip codes 
''' LOOP'''
 elif is_postcode(tag):
     if len(tag.attrib['v']) !=5:
         tag.set('v', tag.attrib['v'][0:5])
```
4. **I don't know how to remove the strange zip codes; a whole node.**

---

# Sqlite3 Work

In [7]:
import sqlite3
import sys
from tabulate import tabulate # For nice representation of the results

## Zip Code Counts

In [11]:
# Opening the file
conn  = sqlite3.connect('Cincinnati')
c = conn.cursor()

# Here I need to combine the ways and nodes table then get from that the values where the key is named 'postcode'
c.execute("SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) \
tags WHERE tags.key='postcode' GROUP BY tags.value ORDER BY count DESC LIMIT 5 ;")
# Delete the limit to see all the zip codes and make sense out of the Observation section.

post = c.fetchall()
Top_post=[]
for row in post:
    Top_post.append(list(row))
print tabulate(Top_post, headers=['Zip Code', 'Count'], tablefmt='fancy_grid')



╒════════════╤═════════╕
│   Zip Code │   Count │
╞════════════╪═════════╡
│      45202 │     401 │
├────────────┼─────────┤
│      45219 │      69 │
├────────────┼─────────┤
│      45208 │      63 │
├────────────┼─────────┤
│      45220 │      28 │
├────────────┼─────────┤
│      45223 │      19 │
╘════════════╧═════════╛


## Top 10 Contributors

In [12]:
# Here I need to combine the nodes and ways tables and get from that the number of time each user appeared.
c.execute("SELECT u.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) u GROUP \
BY u.user ORDER BY num DESC LIMIT 10;")
user_cont =[]

rows = c.fetchall()
for row in rows:
    user_cont.append(list(row))
    
print tabulate(user_cont, headers=['Name', 'Contributtions'], tablefmt='fancy_grid')


╒═════════════════╤══════════════════╕
│ Name            │   Contributtions │
╞═════════════════╪══════════════════╡
│ Minh Nguyen     │           244760 │
├─────────────────┼──────────────────┤
│ Nate_Wessel     │            94439 │
├─────────────────┼──────────────────┤
│ lrhill          │            40176 │
├─────────────────┼──────────────────┤
│ woodpeck_fixbot │            29686 │
├─────────────────┼──────────────────┤
│ MichaelGSmith   │             8768 │
├─────────────────┼──────────────────┤
│ reportingsjr    │             8733 │
├─────────────────┼──────────────────┤
│ gmensch         │             7918 │
├─────────────────┼──────────────────┤
│ Chris Davis     │             4969 │
├─────────────────┼──────────────────┤
│ lightbulbsrwarm │             2997 │
├─────────────────┼──────────────────┤
│ Robin Daugherty │             1265 │
╘═════════════════╧══════════════════╛


## Some Statistics

In [13]:
# Ways
c.execute("SELECT COUNT(*) FROM ways;")
ways = c.fetchall()
Stat=[]
Stat.append(['Ways',ways[0][0]])

# Nodes
c.execute("SELECT COUNT(*) FROM nodes;")
nodes = c.fetchall()
Stat.append(['Nodes',nodes[0][0]])

#Users, to count the number of user I need to count the number of unique uid from nodes and ways tables.
c.execute("SELECT COUNT(DISTINCT(id.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) id;")
Num_users = c.fetchall()
Stat.append(['Users',Num_users[0][0]])

print tabulate(Stat, headers=['Object', '#'], tablefmt='fancy_grid')



╒══════════╤════════╕
│ Object   │      # │
╞══════════╪════════╡
│ Ways     │  70361 │
├──────────┼────────┤
│ Nodes    │ 387282 │
├──────────┼────────┤
│ Users    │    331 │
╘══════════╧════════╛


## Amenities Count

In [14]:
# Here I need to get the values from node_tags table where the key is named 'amenity'
c.execute("SELECT value, COUNT(*) as num \
FROM nodes_tags \
WHERE key='amenity' \
GROUP BY value \
ORDER BY num DESC \
LIMIT 10;")
amenites = c.fetchall()
Amenity=[]
for row in amenites:
    Amenity.append(list(row))
print tabulate(Amenity, headers=['Amenity', '#'], tablefmt='fancy_grid')


╒═══════════╤═════╕
│ Amenity   │   # │
╞═══════════╪═════╡
│ bicyc     │ 131 │
├───────────┼─────┤
│ place     │ 127 │
├───────────┼─────┤
│ resta     │ 121 │
├───────────┼─────┤
│ bench     │ 110 │
├───────────┼─────┤
│ drink     │  90 │
├───────────┼─────┤
│ cafe      │  60 │
├───────────┼─────┤
│ fast_     │  43 │
├───────────┼─────┤
│ schoo     │  43 │
├───────────┼─────┤
│ fuel      │  28 │
├───────────┼─────┤
│ parki     │  26 │
╘═══════════╧═════╛


** I see that the names are not complete and could be improved by editing them. However, I think that is done to save some space, I would leave them as they are; they can be understood and are consistent. **

---

# Observation
 Something interesting I found in the count of the zip codes. While I was expecting that the count of each zip code will reflect the number of people living there, it was not! The top two zip codes are in fact has really low population compared to the rest of the zip codes.
My guessing is that the age and education is the hidden factor here; the count of the zip codes reflect the age distribution and the education there. To see if I'm right, I went to [zipatlas.com](http://zipatlas.com/us/oh/cincinnati.htm) to see the location and the age distribution of some of the zip codes.

I was right. The top two zip codes, [45202](http://zipatlas.com/us/oh/cincinnati/zip-45202.htm) and [45219](http://zipatlas.com/us/oh/cincinnati/zip-45219.htm), are located near [University of Cincinnati](https://goo.gl/maps/Ph5UJZbxa4p) (UC) and most people are between 20 and 30 in age, and I believe that most of them were once studying at UC. On the other hand, [45211](http://zipatlas.com/us/oh/cincinnati/zip-45211.htm), the most populated zip code, is far away from UC and most people are over 30 in age.

I am saying this because bothering about contributing to OSM needs someone who is first aware that OSM even exists and second can deal with it.

It is really hard to think of a solution for this unbalance. I might say that the contributors get extra credit if they put information about the locations that are important but has little information.


### Implementing the solution
* **Benefit**
    * The data are clearly not even close to complete and encouraging more contributions will help in completing the data.
   
* **Difficulties**
   * Obviously you can not contribut unless you know what are you doing, and people usually don't know alot about further location.
   * Some, or many, of those who could contribute are either foreign students or comming from different states, so they don't know much about the far locations from UC.
   
---

## File sizes

* Cincinnati.xml  ---> 89.6MB  <-- (The raw file)
* Cincinnati.db   ---> 61.5MB <-- (The SQL file)
* nodes.csv      -------> 32.7MB
* nodes_tags.csv --> 674KB
* ways.csv     ---------> 4.3MB
* ways_tags.csv  -----> 5.6MB
* ways_nodes.cv  -----> 11.8MB
